*This is a Jupyter Notebook. It is an interactive document that contains both rich text elements such as figures, links, equations, etc. and executable code - in this case Python code (the grey boxes).
**How to use a Jupyter Notebook**: You can execute the blocks of code one at the time by placing the mouse in the grey box and pressing shift + enter. An asterisk will appear in the brackets at the top left of the box while the code is being executed (this may take few seconds) and turns into a number when the execution is over.*

*This notebook and underpinning code were developed by Francesca Pianosi (francesca.pianosi@bristol.ac.uk) and are distributed under the GPL 3.0 licence: https://www.gnu.org/licenses/gpl-3.0.html*


# GSA tutorial - Hydrological modelling example
Francesca Pianosi

In this Notebook we show how Global Sensitivity Analysis (GSA) can be used us to investigate the relative importance of model parameters on a range of model outputs, and therefore identify the most influential parameters on which estimation efforts should be focused on.

## The rainfall-runoff GR6J model
We will apply GSA to a **lumped hydrological model**, which takes time series of meteorological forcings (rainfall and temperature) over a catchment, and returns the time series of river flows at the catchment outlet. Here we will use the GR6J model (Pushpalatha et al 2011), which has six parameters:
- X1: production store capacity [mm]
- X2: intercatchment exchange coefficient [mm/d]
- X3: routing store capacity [mm]
- X4: unit hydrograph time constant [d]
- X5: intercatchment exchange threshold [-]
- X6: exponential store depletion coefficient [mm]

## The performance metrics
To measure the model ability to reproduce the flow observations, we will use three **performance metrics** (all to be minimised):
- Correlation: $ \ \ \ C = ( r - 1 )^2 \ \ $ where $ \ \ \ r $ is the correlation between the observed flow time series ($Q^{obs}$) and the simulated flow time series ($Q^{sim}$)
- Variability: $ \ \ \ V = ( S_{sim} / S_{obs} - 1 )^2 \ \ $ where $S_{sim}$ ($S^{obs}$) is the standard deviation of the time series $Q^{sim}$ ($Q^{obs}$).
- Bias: $ \ \ \ B =  ( M_{sim} / M_{obs} - 1 )^2 \ \ $ where $M^{sim}$ ($M^{obs}$) is the mean of $Q^{sim}$ ($Q^{obs}$)

We will also use a fourth metric, which is widely used in hydrology under the name of King-Gupta Efficiency (https://en.wikipedia.org/wiki/Kling%E2%80%93Gupta_efficiency), and is a combination of C, V and B:

- King-Gupta Efficiency: $ \ \ \ KGE = 1 - \sqrt{ C + V + B } $

By definition, KGE varies between $- \infty $ and 1, where 1 is the value that would be achieved by a perfect model with no error in correlation, variability and bias. Differently from C, V and B thus, the KGE is to be maximised.

We will use time series of precipitation, potential evaporation, and river flows from the CAMELS-GB (Coxon et al. 2020) of an example catchment: https://nrfa.ceh.ac.uk/data/station/spatial/54025.


### Import libraries
Before starting, we need to import some packages

In [1]:
from __future__ import division, absolute_import, print_function

# General Python Packages needed for calculation
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

# Specific functions needed for model simulation and calibration
from GR6J import gr6j_sim as gr6j_sim
from PerfMetrics import CORR_ERR as CORR_ERR
from PerfMetrics import MEAN_ERR as MEAN_ERR
from PerfMetrics import STD_ERR as STD_ERR

# Install SAFE package and import required functions
%pip install SAFEpython
import safepython.PAWN as PAWN # Module to calculate PAWN sensitivity indices
import safepython.plot_functions as pf # Module to visualize the results
import safepython.RSA_thres as RSA_tr # Module that implements RSA
from safepython.model_execution import model_execution # Module to execute the model
from safepython.sampling import AAT_sampling, AAT_sampling_extend # Functions to perform the input sampling
from safepython.util import aggregate_boot # Functions to perform bootstrapping


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 1.9 MB/s eta 0:00:00


### Load rainfall, evap and flow data

In [ ]:
# Load the data:
data = pd.read_csv("CAMELS_GB_hydromet_timeseries_54025_19701001-20150930.csv")
rain = data["precipitation"].to_numpy() # (mm/day)
evap = data["pet"].to_numpy() # (mm/day)
flow = data["discharge_spec"].to_numpy() # (mm/day)

# Select a sub-period of the available time series for further analysis:
rain_sel = rain[280:280+364*3] #
evap_sel = evap[280:280+364*3]
flow_sel = flow[280:280+364*3]

# Plot:
plt.figure(figsize=[15,7])
plt.subplot(311); plt.plot(rain_sel,'g'); plt.ylabel('rainfall (mm/day)')
plt.subplot(312); plt.plot(evap_sel,'g'); plt.ylabel('evaporation (mm/day)')
plt.subplot(313); plt.plot(flow_sel,'g'); plt.ylabel('flow (mm/day)')
plt.xlabel('time (days)')

plt.show()

## 1 - One-At-the-Time Sensitivity Analysis
We are now ready to run the GR6J model. To do this, we will set the six model parameters to some tentative values, run the model and plot the resulting streamflow time series. We can then change the parameter values one-at-a-time and look at how this changes the model predictions, and their fit to observed flows.

Use the following cell to define the parameter values:

In [3]:
x1 = 200 # must vary between 0 and 400
x2 = 0.3 # must vary between 0 and 3
x3 = 100 # must vary between 10 and 150
x4 = 3 # must vary between 0 and 10
x5 = 0.6 # must vary between 0.1 and 1
x6 = 80 # must vary between 0.1 and 100

... and execute the following cell to run the model and plot the simulation results

In [ ]:
# Run simulation:
x = np.array([x1, x2, x3, x4, x5, x6])
flow_sim, _, _ = gr6j_sim(x, rain_sel, evap_sel, np.zeros((3, )))
C = CORR_ERR(flow_sim, flow_sel)
B = MEAN_ERR(flow_sim, flow_sel)
V = STD_ERR(flow_sim, flow_sel)
KGE = 1-(C+B+V)**0.5
# Plot hydrograph of observed and simulated flow:
fig = plt.figure(figsize=[12,3])
plt.plot(flow_sel, 'g') # observed flow
plt.plot(flow_sim, 'k') # simulated flow
plt.ylabel('flow (mm/day)')
plt.xlabel('time (days)')
plt.legend(['obs', 'sim'])
plt.title("Correlation: C = %0.3f" % C + ", Variability: V = %0.3f" % V + ", Bias: B = %0.3f" % B + ", KGE = %0.3f" % KGE, loc='center')
plt.show()


### Questions:
* What happens if you go back to the definition of the parameters and change one of them?
* What parameter controls what aspect of the hydrograph (magnitude and timing of the flow peaks, spead of the recession curve, minimum flow value, etc.)?
* How does the change in hydrograph reflect into change in performance metric?
* Can you find a parameter combination that delivers a good fit to the observations?

##  2 - Global Sensitivity Analysis
In this section, we run Monte Carlo simulations of the model against a prescribed number of parameter combinations, randomly drawn from the feasible parameter ranges. Each model simulation provides a times series of streamflow predictions. For each time series, we will measure the distance from observations through different performance metrics. We can then analyse the sensitivity of these metrics to the different parameters.

###  Run Monte Carlo simulations

In order to run Monte Carlo simulations, we need to make few choices: 1) the parameter ranges from which parameter combinations will be randomly sampled; 2) the number of parameter combinations to be sampled. This is done in the next cell (change the code if you want to make different choices!)

In [5]:
# Define variability ranges of the parameters:
xmin = [0  , 0,   10,  0, 0.1, 0.1  ] # min values
xmax = [300, 0.5,  50, 10, 0.5, 50  ] # max values
# Choose number of parameter combinations to be sampled:
N = 200

We are now ready to run the Monte Carlo simulations (this may take few seconds if you have selected a large value of N) and plot the resulting simulated time series.

In [ ]:
M = len(xmin) # Number of parameters
samp_strat = 'lhs' # sampling strategy
distr_fun = st.uniform # Parameter distributions
# Save lower and upper bound in the appropriate format to be passed on to the sampling function:
distr_par = [np.nan] * M
for i in range(M):
    distr_par[i] = [xmin[i], xmax[i] - xmin[i]]
X = AAT_sampling(samp_strat, M, distr_fun, distr_par, N)
# Execute the model against all the input samples in 'X':
QQ = model_execution(gr6j_sim, X, rain_sel, evap_sel, np.zeros((3, )))
# Plot Monte Carlo (MC) simulations results and compare with data:
plt.figure(figsize=[15,3])
plt.plot(flow_sel, color='g') # plot for legend
plt.plot(np.transpose(QQ), 'grey', linewidth = 0.1)
plt.plot(flow_sel, color='g')
plt.ylabel('flow (mm/day)'); plt.xlabel('time (days)')
plt.legend(['obs', 'sim'])
plt.title("Number of parameter samples: N = %d" % N, loc='right')
plt.show()

### Questions:
* Does the envelope of simulated time series (grey) include the time series of observed flows?
* If not, can you make that happen by expanding the parameter ranges and/or increasing the number of samples (N)?
* *Advanced hydrology question: What other sources of errors may cause the mismatch between observations and simulations?*

###  Produce scatter plots

We can now calculate the performance metrics for each of the simulated time series and the produce scatter plots showing how performance varies against paramter values.

Use the following cell to choose which performance metric to look at (you can comment all options using # and uncomment the one you chose):

In [8]:
metric = 'CORRELATION'
#metric = 'BIAS'
#metric = 'VARIABILITY'
#metric = 'KGE'

... and execute the following cell to produce the scatter plots

In [ ]:
YY = np.nan * np.ones((N, 4))
YY[:, 0] = CORR_ERR(QQ, flow_sel)
YY[:, 1] = MEAN_ERR(QQ, flow_sel)
YY[:, 2] = STD_ERR(QQ, flow_sel)
YY[:, 3] = 1 - ( YY[:, 0]+YY[:, 1]+YY[:, 2] )**0.5
X_Labels = ['x1', 'x2', 'x3', 'x4', 'x5','x6'] # Name of parameters (used to customize plots)
Y_Labels = ['CORRELATION','BIAS','VARIABILITY','KGE']
if metric == 'CORRELATION':
    j = 0
elif metric == 'BIAS':
    j = 1
elif metric == 'VARIABILITY':
    j = 2
elif metric == 'KGE':
    j = 3
plt.figure(figsize=[18,3])
for i in range(M):
    plt.subplot(1,M,i+1)
    plt.plot(X[:, i], YY[:,j], '.', markerfacecolor="white", markeredgecolor="grey")
    if i == 0:
        plt.ylabel(Y_Labels[j])
    else:
        plt.yticks([])
    plt.xlabel(X_Labels[i])
plt.show()

### Questions:
* From these scatter plots, which parameter would you say is most influential? Why?
* How does the answer changes with the chosen performance metric?
* *Advanced hydrology question: Can you interpret why certain parameters are more important for a certain metric than others?*

### Calculate sensitivity indices with PAWN
In this section, we formally assess the sensitivity of the performance metrics to the model parameters through the PAWN method (Pianosi and Wagener, 2018). The analysis can be repeated for each performance metric. For each metric, we can also assess the impact of the choice of the tuning parameters of the PAWN method: the number of conditioning interval (n), the aggregation statistic (median, mean, max) and the number of bootstrap resamples (Nboot) used to estimate confidence intervals of the PAWN indices.

In [ ]:
Y_Labels = ['CORRELATION','BIAS','VARIABILITY','KGE']
P=len(Y_Labels)


# Choose the PAWN Tuning parameters:
# n = number of conditioning intervals
# aggr = statistic to aggregate KS values
# Nboot = number of bootstrapping resamples used to derive confidence bounds of sensitivity indices
n = 5 # vary between 3 and 15
aggr = 'mean' # choose from 'mean','median' and 'max'
Nboot = 50 # vary between 5 and 100

plt.figure(figsize=[18,3])
for j in range(P):
    # Extract output metric:
    Y = YY[:, j];
    # Compute sensitivity indices for Nboot bootstrap resamples
    KS_median, KS_mean, KS_max = PAWN.pawn_indices(X, Y, n, Nboot=Nboot) # shape (Nboot, M)
    # Compute mean and confidence intervals of the sensitivity indices across the bootstrap resamples:
    KS_median_m, KS_median_lb, KS_median_ub = aggregate_boot(KS_median) # shape (M,)
    KS_mean_m, KS_mean_lb, KS_mean_ub = aggregate_boot(KS_mean) # shape (M,)
    KS_max_m, KS_max_lb, KS_max_ub = aggregate_boot(KS_max) # shape (M,)
    # Plot sensitivity indices:
    plt.subplot(1,4,j+1)
    fs = 10
    plt.title("Output metric = %s" % Y_Labels[j],fontsize=fs)
    if aggr == 'median':
        pf.boxplot1(KS_median_m, S_lb=KS_median_lb, S_ub=KS_median_ub, X_Labels=X_Labels)
    if aggr == 'mean':
        pf.boxplot1(KS_mean_m, S_lb=KS_mean_lb, S_ub=KS_mean_ub, X_Labels=X_Labels)
    if aggr == 'max':
        pf.boxplot1(KS_max_m, S_lb=KS_max_lb, S_ub=KS_max_ub, X_Labels=X_Labels)
    if j > 0:
        plt.ylabel('');
    else:
        plt.ylabel('sensitivity',fontsize=fs)
    plt.xticks(fontsize=fs)
    plt.yticks(fontsize=fs)
plt.show()

### Questions:
* Which parameter are most influential on each performance metric?
* Are these results consistent with the visual inspection of the scatter plots?
* *Advanced GSA question: What is the impact of changing n and Nboot? Why?*

### Calculate sensitivity indices with Regional Sensitivity Analysis
In this section, we repeat the GSA using another method: Regional Sensitivity Analysis (Spear and Hornberger, 1980). We do this to show that multiple approaches are available to perform GSA, but also because the Regional SA approach is tightly connected with the GLUE approach (Beven and Binley 2013) for parameter estimation. For each metric, we need to choose a threshold value to distinguish between 'behavioural' parameter combinations (=those for which the model meets the performance threshold) and 'non-behavioural'. Sensitivity is then measured by the vertical distance between the cumulative distribution of the 'behavioural' parameters and that of the 'non-behavioural'.

Use the following cell to choose which performance metric to look at and the associated threshold value:

In [20]:
metric = 'CORRELATION'
#metric = 'BIAS'
#metric = 'VARIABILITY'
#metric = 'KGE'
threshold = 0.1 # suggest to vary between 0.02 and 0.1

... and execute the following cell to run the Regional Sensitivity Analysis

In [ ]:
YY = np.nan * np.ones((N, 4))
YY[:, 0] = CORR_ERR(QQ, flow_sel)
YY[:, 1] = MEAN_ERR(QQ, flow_sel)
YY[:, 2] = STD_ERR(QQ, flow_sel)
X_Labels = ['x1', 'x2', 'x3', 'x4', 'x5','x6'] # Name of parameters (used to customize plots)
Y_Labels = ['CORRELATION','BIAS','VARIABILITY']

if metric == 'CORRELATION':
    j = 0
elif metric == 'BIAS':
    j = 1
elif metric == 'VARIABILITY':
    j = 2
mvd, _, _, idx = RSA_tr.RSA_indices_thres(X, YY[:, j], threshold)
plt.figure(figsize=[18,7])
for i in range(M):
    plt.subplot(2,M,i+1)
    plt.plot(X[idx, i], YY[idx,j], '.', markerfacecolor="blue", markeredgecolor="grey")
    plt.plot(X[~idx, i], YY[~idx,j], '.', markerfacecolor="white", markeredgecolor="grey")
    plt.legend(['behavioural','non-behavioural'],loc="upper left")
    plt.xticks([])
    if i == 0:
        plt.ylabel(Y_Labels[j])
    else:
        plt.yticks([])
    plt.subplot(2,M,M+i+1)
    plt.ecdf(X[idx, i],color='blue')
    plt.ecdf(X[~idx, i],color='grey')
    plt.xlabel(X_Labels[i])
    plt.title("Sensitivity = %0.2f" %mvd[i] )
    if i == 0:
        plt.ylabel('CDF')
    else:
        plt.yticks([])

plt.show()



### Questions:
* Are sensitivity results consistent with the PAWN method?
* What other interesting information we can get out of the CDF (Cumulative Distribution Function) plots of the behavioural parameters?

### References

Beven and Binley (2013). GLUE: 20 years on, Hyd. Proc. 28(24), 5897-5918.

Coxon et al. (2020). CAMELS-GB: hydrometeorological time series and landscape attributes for 671 catchments in Great Britain, Earth Syst. Sci. Data, 12, 2459–2483.

Pianosi et al. (2015). A Matlab toolbox for Global Sensitivity Analysis’. Env. Mod. & Soft., 70, 80-85.

Pianosi and Wagener (2018). Distribution-based sensitivity analysis from a generic input-output sample, Env. Mod. & Soft., 108, 197-207.

Pushpalatha, R., Perrin, C., Le Moine, N., Mathevet, T. and Andréassian, V. (2011). A downward structural sensitivity analysis of hydrological models to improve low-flow simulation. Journal of Hydrology, 411(1-2), 66-76, doi:10.1016/j.jhydrol.2011.09.034.

Spear and Hornberger (1980). Eutrophication in peel inlet. II. Identification of critical uncertainties via generalized sensitivity analysis, Water Res., 14, 43-49.

